In [1]:
import polars as pl
import duckdb
from rank_bm25 import BM25Okapi
import timeit
import bm25s

/home/naabzer/develop/recipe_tm/env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [39]:
df = duckdb.sql("SELECT * FROM 'eval_test.parquet'").pl()

In [67]:
start = timeit.default_timer()
corpus = list(df['tokens'].slice(0,100))
bm25 = BM25Okapi(corpus)
bm25.get_scores("red_onion butter olive_oil".split(" "))
time = timeit.default_timer() - start

In [68]:
print(time)

0.011778720014262944


array([0.        , 0.        , 1.17074143, ..., 0.        , 0.        ,
       1.56839375], shape=(10000,))

In [30]:
corpus = [" ".join(tokens) for tokens in df['tokens']]
start = timeit.default_timer()
bm25_tokens = bm25s.tokenize(corpus)
time = timeit.default_timer() - start

In [31]:
print(time)

0.14562488498631865


In [32]:
start = timeit.default_timer()
retriever = bm25s.BM25()
retriever.index(bm25_tokens)
time = timeit.default_timer() - start

In [33]:
time

0.47601976996520534

In [34]:
query_t = bm25s.tokenize("red_onion butter olive_oil")
results, scores = retriever.retrieve(query_t, corpus=corpus)

In [35]:
results

array([['olive_oil butter red_onion balsamic_vinegar salt',
        'red_onion olive_oil balsamic_vinegar honey black_pepper',
        'cucumber feta red_onion dijon_mustard olive_oil sugar',
        'red_onion cooking_spray olive_oil balsamic_vinegar salt black_pepper',
        'orange red_onion olive salt pepper basil olive_oil tbsp',
        'beet olive_oil lemon_juice vinegar honey salt pepper grapefruit red_onion',
        'potato salt parsley red_onion cherry_pepper chive sweet_relish caper olive_oil salt',
        'flour milk egg salt aromatic_cheese red_onion butter pepper chive',
        'pasta tightly_pack_tear chicken tomato red_onion italian_dressing',
        'gratin olive_oil red_onion garlic salt flour chickpea spinach breadcrumb olive_oil garlic smoke_paprika salt bread']],
      dtype='<U114')